In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform google-genai datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.3/171.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.


In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [2]:
import os

from google import genai
from google.genai import types

PROJECT_ID = "baidao-test-666808"  # @param {type:"string", isTemplate: true}
if PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Import libraries

In [3]:
from collections import Counter
import json
import random

# Vertex AI SDK
from google.cloud import aiplatform
from google.cloud.aiplatform.metadata import context
from google.cloud.aiplatform.metadata import utils as metadata_utils
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import vertexai
from datasets import load_dataset
import json
import random
import pandas as pd

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
# Provide a bucket name
BUCKET_NAME = "sql-create-context"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"
print(BUCKET_URI)

gs://sql-create-context


In [5]:
# 加载数据集
ds = load_dataset("b-mc2/sql-create-context")

# 获取训练集数据
data = ds['train']

# 首先将数据分成 train 和 remaining (9:1)
train_test = data.train_test_split(test_size=0.2, seed=42)
train_data = train_test['train']

# 将 remaining 数据再次分成 valid 和 test (1:1)
remaining = train_test['test'].train_test_split(test_size=0.5, seed=42)
valid_data = remaining['train']
test_data = remaining['test']

# 将数据集转换为DataFrame
train_df = pd.DataFrame(train_data)
valid_df = pd.DataFrame(valid_data)
test_df = pd.DataFrame(test_data)

README.md:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [6]:
train_df = train_df.sample(frac=0.1, random_state=42)
valid_df = valid_df.sample(frac=0.1, random_state=42)
test_df = test_df.sample(frac=0.1, random_state=42)

# 打印抽样后的DataFrame大小
print(f"Sampled train DataFrame size: {len(train_df)}")
print(f"Sampled validation DataFrame size: {len(valid_df)}")
print(f"Sampled test DataFrame size: {len(test_df)}")

Sampled train DataFrame size: 6286
Sampled validation DataFrame size: 786
Sampled test DataFrame size: 786


In [7]:
test_df.head(2)

,answer,question,context
7093,SELECT rating___percentage_ FROM table_name_73...,Name the rating % for channel of anhui satelli...,CREATE TABLE table_name_73 (rating___percentag...
1405,SELECT us_viewers__millions_ FROM table_222652...,How many million U.s. Viewers watched episode ...,CREATE TABLE table_22265225_1 (us_viewers__mil...


In [8]:
row_dataset = random.randint(0, 100)

In [9]:
few_shot_examples = test_df.sample(3)
dropped_indices = few_shot_examples.index
test_df = test_df.drop(dropped_indices)

few_shot_prompt = ""
for _, row in few_shot_examples.iterrows():
    few_shot_prompt += (
        f"Context: {row.context}\nQuestion: {row.question}\nAnswer: {row.answer}\n\n"
    )

print(few_shot_prompt)

Context: CREATE TABLE table_name_74 (popular_votes VARCHAR, office VARCHAR, year VARCHAR)
Question: Name the total number of popular votes for mn attorney general in 1994
Answer: SELECT COUNT(popular_votes) FROM table_name_74 WHERE office = "mn attorney general" AND year = 1994

Context: CREATE TABLE table_name_80 (year INTEGER, attendance INTEGER)
Question: How many years have less than 28,505 attendance?
Answer: SELECT AVG(year) FROM table_name_80 WHERE attendance < 28 OFFSET 505

Context: CREATE TABLE table_name_28 (country VARCHAR, winter_olympics VARCHAR)
Question: Which Country has a Winter Olympics of 1948?
Answer: SELECT country FROM table_name_28 WHERE winter_olympics = "1948"




In [10]:
systemInstruct = f"""You are a Database Query Assistant. Your role is to generate SQL queries based on user questions and the provided context. The context will be provided in the form of CREATE TABLE statements that define the database schema.\n\n
Here are some examples: \n\n
{few_shot_prompt}"""

In [11]:
test_df["systemInstruct"] = systemInstruct

test_df["input_question"] = (
    "\n\n**Below the question with context that you need to answer**"
    + "\nContext: " + test_df["context"]
    + "\nQuestion: " + test_df["question"]
)

test_systemInstruct = test_df["systemInstruct"].iloc[row_dataset]
print(test_systemInstruct)
test_question = test_df["input_question"].iloc[row_dataset]
print(test_question)

You are a Database Query Assistant. Your role is to generate SQL queries based on user questions and the provided context. The context will be provided in the form of CREATE TABLE statements that define the database schema.


Here are some examples: 


Context: CREATE TABLE table_name_74 (popular_votes VARCHAR, office VARCHAR, year VARCHAR)
Question: Name the total number of popular votes for mn attorney general in 1994
Answer: SELECT COUNT(popular_votes) FROM table_name_74 WHERE office = "mn attorney general" AND year = 1994

Context: CREATE TABLE table_name_80 (year INTEGER, attendance INTEGER)
Question: How many years have less than 28,505 attendance?
Answer: SELECT AVG(year) FROM table_name_80 WHERE attendance < 28 OFFSET 505

Context: CREATE TABLE table_name_28 (country VARCHAR, winter_olympics VARCHAR)
Question: Which Country has a Winter Olympics of 1948?
Answer: SELECT country FROM table_name_28 WHERE winter_olympics = "1948"




**Below the question with context that you need 

In [12]:
base_model = "gemini-2.0-flash-lite-001"

In [13]:
def get_predictions(question: str, model_version: str) -> str:

    prompt = question
    base_model = model_version

    response = client.models.generate_content(
        model=base_model,
        contents=prompt,
        config={
            "system_instruction": systemInstruct,
            "temperature": 0.3,
        },
    )

    return response.text

In [14]:
test_answer = test_df["answer"].iloc[row_dataset]
response = get_predictions(test_question, base_model)

print(f"Gemini response: {response}")
print(f"Actual answer: {test_answer}")

Gemini response: Answer: SELECT min(w) FROM table_25933764_1 WHERE locale = "Ontario"

Actual answer: SELECT MIN(w) FROM table_25933764_1 WHERE locale = Ontario


In [23]:
from tqdm import tqdm
tqdm.pandas()

test_df["predicted_answer"] = test_df["input_question"].progress_apply(lambda x: get_predictions(x, base_model))
test_df.head(2)

100%|██████████| 783/783 [05:18<00:00,  2.46it/s]


,answer,question,context,systemInstruct,input_question,predicted_answer
7093,SELECT rating___percentage_ FROM table_name_73...,Name the rating % for channel of anhui satelli...,CREATE TABLE table_name_73 (rating___percentag...,You are a Database Query Assistant. Your role ...,\n\n**Below the question with context that you...,Answer: SELECT rating___percentage_ FROM table...
1405,SELECT us_viewers__millions_ FROM table_222652...,How many million U.s. Viewers watched episode ...,CREATE TABLE table_22265225_1 (us_viewers__mil...,You are a Database Query Assistant. Your role ...,\n\n**Below the question with context that you...,Answer: SELECT us_viewers__millions_ FROM tabl...


In [15]:
def f1_score_squad(prediction, ground_truth):
    prediction_tokens = prediction.split()
    ground_truth_tokens = ground_truth.split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return prediction == ground_truth


def calculate_em_and_f1(y_true, y_pred):
    """Calculates EM and F1 scores for DataFrame columns."""

    # Ensure inputs are Series
    if not isinstance(y_true, pd.Series):
        y_true = pd.Series(y_true)
    if not isinstance(y_pred, pd.Series):
        y_pred = pd.Series(y_pred)

    em = np.mean(y_true.combine(y_pred, exact_match_score))
    f1 = np.mean(y_true.combine(y_pred, f1_score_squad))

    return em, f1

In [24]:
em, f1 = calculate_em_and_f1(test_df["answer"], test_df["predicted_answer"])
print(f"EM score: {em}")
print(f"F1 score: {f1}")

EM score: 0.002554278416347382
F1 score: 0.7726125323166296


In [17]:
train_df["input_question"] = (
    "\n\n**Below the question with context that you need to answer**"
    + "\nContext: "
    + train_df["context"]
    + "\nQuestion: "
    + train_df["question"]
)
valid_df["input_question"] = (
    "\n\n**Below the question with context that you need to answer**"
    + "\nContext: "
    + valid_df["context"]
    + "\nQuestion: "
    + valid_df["question"]
)

In [18]:
train_df.head(2)

,answer,question,context,input_question
22529,SELECT release_date FROM table_name_92 WHERE l...,What is the release date of the 6 mb L3 cache ...,CREATE TABLE table_name_92 (release_date VARCH...,\n\n**Below the question with context that you...
21363,"SELECT T2.guest_first_name, T1.booking_start_d...","Show the guest first names, start dates, and e...",CREATE TABLE Apartment_Bookings (booking_start...,\n\n**Below the question with context that you...


In [19]:
def df_to_jsonl(df, output_file):
    """Converts a Pandas DataFrame to JSONL format and saves it to a file.

    Args:
      df: The DataFrame to convert.
      output_file: The name of the output file.
    """

    with open(output_file, "w") as f:
        for row in df.itertuples(index=False):
            jsonl_obj = {
                "systemInstruction": {"parts": [{"text": f"{systemInstruct}"}]},
                "contents": [
                    {
                        "role": "user",
                        "parts": [{"text": f"{row.input_question}"}],
                    },
                    {"role": "model", "parts": [{"text": row.answer}]},
                ],
            }
            f.write(json.dumps(jsonl_obj) + "\n")


# Process the DataFrames
df_to_jsonl(train_df, "train.jsonl")
df_to_jsonl(valid_df, "valid.jsonl")

print(f"JSONL data written to train.jsonl")
print(f"JSONL data written to valid.jsonl")

JSONL data written to train.jsonl
JSONL data written to valid.jsonl


Next you will copy the files into your Google Cloud bucket

In [20]:
!gsutil cp ./train.jsonl {BUCKET_URI}
!gsutil cp ./valid.jsonl {BUCKET_URI}

Copying file://./train.jsonl [Content-Type=application/octet-stream]...
/ [1 files][  8.5 MiB/  8.5 MiB]                                                
Operation completed over 1 objects/8.5 MiB.                                      
Copying file://./valid.jsonl [Content-Type=application/octet-stream]...
/ [1 files][  1.1 MiB/  1.1 MiB]                                                
Operation completed over 1 objects/1.1 MiB.                                      


In [21]:
train_dataset = f"""{BUCKET_URI}/train.jsonl"""
validation_dataset = f"""{BUCKET_URI}/valid.jsonl"""

training_dataset = {
    "gcs_uri": train_dataset,
}

validation_dataset = types.TuningValidationDataset(gcs_uri=validation_dataset)

In [22]:
sft_tuning_job = client.tunings.tune(
    base_model=base_model,
    training_dataset=training_dataset,
    config=types.CreateTuningJobConfig(
        adapter_size="ADAPTER_SIZE_EIGHT",
        epoch_count=1,  # set to one to keep time and cost low
        tuned_model_display_name="sql-Tuned",
        validation_dataset=validation_dataset,
    ),
)
sft_tuning_job

<ipython-input-22-e094f175d366>:1: ExperimentalWarning: The SDK's tuning implementation is experimental, and may change in future versions.
  sft_tuning_job = client.tunings.tune(


TuningJob(name='projects/746866758104/locations/us-central1/tuningJobs/8738190145729069056', state=<JobState.JOB_STATE_PENDING: 'JOB_STATE_PENDING'>, create_time=datetime.datetime(2025, 5, 16, 1, 41, 51, 773084, tzinfo=TzInfo(UTC)), start_time=None, end_time=None, update_time=datetime.datetime(2025, 5, 16, 1, 41, 51, 773084, tzinfo=TzInfo(UTC)), error=None, description=None, base_model='gemini-2.0-flash-lite-001', tuned_model=None, supervised_tuning_spec=SupervisedTuningSpec(hyper_parameters=SupervisedHyperParameters(adapter_size=<AdapterSize.ADAPTER_SIZE_EIGHT: 'ADAPTER_SIZE_EIGHT'>, epoch_count=1, learning_rate_multiplier=None), training_dataset_uri='gs://sql-create-context/train.jsonl', validation_dataset_uri='gs://sql-create-context/valid.jsonl', export_last_checkpoint_only=None), tuning_data_stats=None, encryption_spec=None, partner_model_tuning_spec=None, distillation_spec=None, experiment=None, labels=None, pipeline_job=None, tuned_model_display_name='sql-Tuned')

In [27]:
sft_tuning_job.state

<JobState.JOB_STATE_PENDING: 'JOB_STATE_PENDING'>

In [28]:
tuning_job = client.tunings.get(name=sft_tuning_job.name)
tuning_job

TuningJob(name='projects/746866758104/locations/us-central1/tuningJobs/8738190145729069056', state=<JobState.JOB_STATE_SUCCEEDED: 'JOB_STATE_SUCCEEDED'>, create_time=datetime.datetime(2025, 5, 16, 1, 41, 51, 773084, tzinfo=TzInfo(UTC)), start_time=datetime.datetime(2025, 5, 16, 1, 41, 51, 824706, tzinfo=TzInfo(UTC)), end_time=datetime.datetime(2025, 5, 16, 2, 2, 41, 454026, tzinfo=TzInfo(UTC)), update_time=datetime.datetime(2025, 5, 16, 2, 2, 41, 454026, tzinfo=TzInfo(UTC)), error=None, description=None, base_model='gemini-2.0-flash-lite-001', tuned_model=TunedModel(model='projects/746866758104/locations/us-central1/models/7861177185598439424@1', endpoint='projects/746866758104/locations/us-central1/endpoints/7846806568623407104', checkpoints=[TunedModelCheckpoint(checkpoint_id='1', epoch=1, step=298, endpoint='projects/746866758104/locations/us-central1/endpoints/7846806568623407104')]), supervised_tuning_spec=SupervisedTuningSpec(hyper_parameters=SupervisedHyperParameters(adapter_siz

In [29]:
experiment_name = tuning_job.experiment
experiment_name

'projects/746866758104/locations/us-central1/metadataStores/default/contexts/tuning-experiment-20250515184234090711'

In [30]:
# Locate Vertex AI Experiment and Vertex AI Experiment Run
experiment = aiplatform.Experiment(experiment_name=experiment_name)
filter_str = metadata_utils._make_filter_string(
    schema_title="system.ExperimentRun",
    parent_contexts=[experiment.resource_name],
)
experiment_run = context.Context.list(filter_str)[0]

In [31]:
# Read data from Tensorboard
tensorboard_run_name = f"{experiment.get_backing_tensorboard_resource().resource_name}/experiments/{experiment.name}/runs/{experiment_run.name.replace(experiment.name, '')[1:]}"
tensorboard_run = aiplatform.TensorboardRun(tensorboard_run_name)
metrics = tensorboard_run.read_time_series_data()

In [32]:
def get_metrics(metric: str = "/train_total_loss"):
    """
    Get metrics from Tensorboard.

    Args:
      metric: metric name, eg. /train_total_loss or /eval_total_loss.
    Returns:
      steps: list of steps.
      steps_loss: list of loss values.
    """
    loss_values = metrics[metric].values
    steps_loss = []
    steps = []
    for loss in loss_values:
        steps_loss.append(loss.scalar.value)
        steps.append(loss.step)
    return steps, steps_loss

In [33]:
# Get Train and Eval Loss
train_loss = get_metrics(metric="/train_total_loss")
eval_loss = get_metrics(metric="/eval_total_loss")

In [34]:
# Plot the train and eval loss metrics using Plotly python library
fig = make_subplots(
    rows=1, cols=2, shared_xaxes=True, subplot_titles=("Train Loss", "Eval Loss")
)

# Add traces
fig.add_trace(
    go.Scatter(x=train_loss[0], y=train_loss[1], name="Train Loss", mode="lines"),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(x=eval_loss[0], y=eval_loss[1], name="Eval Loss", mode="lines"),
    row=1,
    col=2,
)

# Add figure title
fig.update_layout(title="Train and Eval Loss", xaxis_title="Steps", yaxis_title="Loss")

# Set x-axis title
fig.update_xaxes(title_text="Steps")

# Set y-axes titles
fig.update_yaxes(title_text="Loss")

# Show plot
fig.show()

### Use the fine-tuned model and evaluation

In [35]:
prompt = """
Answer the question based on the context

question: What is the branding for callsign dypv?,
context: CREATE TABLE table_27588823_2 (branding VARCHAR, callsign VARCHAR)
"""

In [36]:
tuned_model = tuning_job.tuned_model.endpoint
tuned_model

'projects/746866758104/locations/us-central1/endpoints/7846806568623407104'

In [37]:
get_predictions(prompt, tuned_model)

'SELECT branding FROM table_27588823_2 WHERE callsign = "DYPV"'

In [38]:
# Apply the get_prediction() function to the 'question_column'
test_df["predicted_answer"] = test_df["input_question"].progress_apply(lambda x: get_predictions(x, tuned_model))
test_df.head(2)

100%|██████████| 783/783 [05:48<00:00,  2.25it/s]


,answer,question,context,systemInstruct,input_question,predicted_answer
7093,SELECT rating___percentage_ FROM table_name_73...,Name the rating % for channel of anhui satelli...,CREATE TABLE table_name_73 (rating___percentag...,You are a Database Query Assistant. Your role ...,\n\n**Below the question with context that you...,SELECT rating___percentage_ FROM table_name_73...
1405,SELECT us_viewers__millions_ FROM table_222652...,How many million U.s. Viewers watched episode ...,CREATE TABLE table_22265225_1 (us_viewers__mil...,You are a Database Query Assistant. Your role ...,\n\n**Below the question with context that you...,SELECT us_viewers__millions_ FROM table_222652...


In [39]:
em, f1 = calculate_em_and_f1(test_df["answer"], test_df["predicted_answer"])
print(f"EM score: {em}")
print(f"F1 score: {f1}")

EM score: 0.7662835249042146
F1 score: 0.967937026393091
